# Participación en fideicomisos con posible conflicto de interés (4_2_09)
| **Escenario**                                                 | **Condición evaluada**                                  | **Resultado**    |
| ------------------------------------------------------------- | ------------------------------------------------------- | ---------------- |
| **No hay fideicomisos declarados**                            | Lista `fideicomisos.fideicomiso` no existe o está vacía | ⚪ **NO_APLICA**  |
| **Faltan datos esenciales**                                   | Sector productivo vacío / nulo                          | ⚪ **SIN_DATO**   |
| **Sector productivo del fideicomisario pertenece a T, D o C** | `sector ∈ sectores_riesgo[funcion_principal_ente]`      | 🔴 **NO_CUMPLE** |
| **Sector productivo NO pertenece a sectores de riesgo**       | Sector fuera de T, D, C                                 | 🟢 **CUMPLE**    |
| **Registros presentes pero incompletos**                      | Declaraciones de fideicomisos sin sector                | ⚪ **SIN_DATO**   |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata


METRIC_ID = "4_2_09_FIDEICOMISOS_SECTOR_RIESGO"


# -------------------------------------------------------------------
# MATRIZ TEMPORAL DE SECTORES (reemplazar con el catálogo real INEGI)
# -------------------------------------------------------------------
MATRIZ_SECTORES_FUNCIONES = {
    "SALUD": ["T", "D"],
    "OBRAS PUBLICAS": ["T", "C"],
    "EDUCACION": ["D"],
    "SEGURIDAD": ["T", "C"],
}


# -------------------------------------------------------------------
# AUXILIARES UNIVERSALES
# -------------------------------------------------------------------
def limpiar(v):
    if not v:
        return None
    v = str(v).strip().upper()
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    while "  " in v:
        v = v.replace("  ", " ")
    return v


def extraer(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None


# -------------------------------------------------------------------
# EVALUACIÓN DE LA MÉTRICA
# -------------------------------------------------------------------
def evaluar_metrica(doc):

    # 1. Nombre del ente público
    ente = limpiar(
        extraer(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico")
    )

    if not ente:
        return "SIN_DATO"

    funcion_principal = ente  # será reemplazado por catálogo real
    sectores_riesgo = MATRIZ_SECTORES_FUNCIONES.get(funcion_principal, [])

    # 2. Fideicomisos declarados
    fideicomisos = extraer(doc, "declaracion.intereses.fideicomisos.fideicomiso")

    if not fideicomisos:
        return "NO_APLICA"

    if not isinstance(fideicomisos, list):
        fideicomisos = [fideicomisos]

    hubo_datos = False

    for f in fideicomisos:
        sector = limpiar(f.get("sector"))

        if not sector:
            return "SIN_DATO"

        hubo_datos = True

        # Regla: sector ∈ riesgos ➜ NO_CUMPLE
        if sector in sectores_riesgo:
            return "NO_CUMPLE"

    if not hubo_datos:
        return "NO_APLICA"

    return "CUMPLE"


# -------------------------------------------------------------------
# PROCESAMIENTO MONGO
# -------------------------------------------------------------------
def procesar_metrica_4_2_09():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision": 1,
            "declaracion.intereses.fideicomisos": 1,
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"⚠️ Error documento {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            # bulk-updates
            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > {total} documentos procesados...")

        # último envío
        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print("Total procesados:", total)
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_2_09()



🚀 Procesando métrica 4_2_09_FIDEICOMISOS_SECTOR_RIESGO...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > 2000 documentos procesados...
 > 4000 documentos procesados...
 > 6000 documentos procesados...
 > 8000 documentos procesados...


Traceback (most recent call last):
  File "C:\Users\Mauro\AppData\Local\Temp\ipykernel_19100\2589699102.py", line 139, in procesar_metrica_4_2_09
    tgt.bulk_write(operaciones)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\_csot.py", line 125, in csot_wrapper
    return func(self, *args, **kwargs)
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py", line 791, in bulk_write
    bulk_api_result = blk.execute(write_concern, session, _Op.INSERT)
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\bulk.py", line 751, in execute
    return self.execute_command(generator, write_concern, session, operation)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\bulk.py", line 604, in execute_command
    _ = client._retryable_write(
        self.is_retryable,
    ...<4 lines>...
        operation_id=op_id

Conexión cerrada.
